# Face Detection Project With DNN OpenCV

https://github.com/opencv/opencv_3rdparty/raw/19512576c112aa2c7b6328cb0e8d589a4a90a26d/res10_300x300_ssd_iter_140000_fp16.caffemodel
https://github.com/opencv/opencv/blob/master/samples/dnn/face_detector

## Load Image And SSD ResNet Caffe Model

In [1]:
import numpy as np
import cv2

In [2]:
img = cv2.imread('faces.jpg')

cv2.imshow('face',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
face_detection_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt',
                                                './models/res10_300x300_ssd_iter_140000_fp16.caffemodel')


## Calculate Blob From Image

In [7]:
# step-1: blob from image
blob = cv2.dnn.blobFromImage(img,1,(300,300),(104,177,123),swapRB=True)
# Size of the image i just want to convert is 300x300, the reason is because my caffe model is
# actually trained with the images of 300x300. That is the reason why we have to resize it to 300x300.

## Get Face Detections Bounding Boxes From the DNN Model

In [8]:
# step-2: set blob as input
face_detection_model.setInput(blob)

# step-3: get the output
detections = face_detection_model.forward()

In [5]:
detections

array([[[[0.        , 1.        , 0.99828076, ..., 0.13435292,
          0.3774765 , 0.6368809 ],
         [0.        , 1.        , 0.93064004, ..., 0.17351156,
          0.66801995, 0.6749055 ],
         [0.        , 1.        , 0.66491354, ..., 0.14685212,
          0.9633989 , 0.5360528 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [6]:
detections.shape

(1, 1, 200, 7)

## Bounding Box - Set the Threshold Confidence Score

In [9]:
detections[0,0,:,2]

array([0.99828076, 0.93064004, 0.66491354, 0.11303048, 0.09906532,
       0.09581406, 0.09539089, 0.09210999, 0.09148152, 0.09004118,
       0.08952264, 0.08925449, 0.08845698, 0.08835823, 0.08817656,
       0.08785023, 0.08761983, 0.08759254, 0.08733169, 0.08644606,
       0.08633808, 0.08580611, 0.08571842, 0.08569968, 0.08565515,
       0.08561672, 0.08543263, 0.08516433, 0.08498956, 0.08490543,
       0.0847211 , 0.08466659, 0.0845129 , 0.084322  , 0.08402869,
       0.08339369, 0.08325931, 0.08319432, 0.0830019 , 0.08287056,
       0.08281701, 0.08280215, 0.08230655, 0.08206578, 0.08128528,
       0.08119465, 0.08109775, 0.08107877, 0.08065075, 0.08063325,
       0.08060352, 0.0803544 , 0.08024481, 0.07988577, 0.07988251,
       0.07974184, 0.07961009, 0.07956649, 0.07941601, 0.07937433,
       0.07934451, 0.07927636, 0.07913712, 0.07913007, 0.07907978,
       0.07904796, 0.07900915, 0.07894583, 0.07892095, 0.07890494,
       0.0788851 , 0.07886859, 0.0788213 , 0.07861337, 0.07850

In [10]:
#  step-4: drawing boundng box on top of face detected
image = img.copy()
h,w = image.shape[:2]
for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2]
    if confidence > 0.5:
        print(confidence)

0.99828076
0.93064004
0.66491354


## Bounding Box - De-Normalize Bounding Box Co-ordinates

In [11]:
image = img.copy()
h,w = image.shape[:2]
for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2]
    if confidence > 0.5:
        # diagonal points 3: 7
        box = detections[0,0,i,3:7]
        print(box) # These boxes are normalized to height and width of the image.

[0.14406163 0.13435292 0.3774765  0.6368809 ]
[0.42392105 0.17351156 0.66801995 0.6749055 ]
[0.75987417 0.14685212 0.9633989  0.5360528 ]


In order to de-normalize that i just need to multiply this with my width and height.

In [12]:
image = img.copy()
h,w = image.shape[:2]
for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2]
    if confidence > 0.5:
        # diagonal points 3: 7
        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        print(box)

[138.29916     94.04704571 362.37745285 445.81661224]
[406.9642067  121.45809531 641.29915237 472.43383527]
[729.47919846 102.79648453 924.86291885 375.23697615]


In [13]:
# OpenCV always suppors the integer values, so we need to change the data type here.
image = img.copy()
h,w = image.shape[:2]
for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2]
    if confidence > 0.5:
        # diagonal points 3: 7
        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        box = box.astype('int')
        print(box)

[138  94 362 445]
[406 121 641 472]
[729 102 924 375]


In [14]:
image = img.copy()
h,w = image.shape[:2]
for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2]
    if confidence > 0.5:
        # diagonal points 3: 7
        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        box = box.astype('int')
        pt1 = (box[0],box[1])
        pt2 = (box[2],box[3])

## Bounding Box - Draw Rectangle And Put Text(Confidence Score)

In [15]:
image = img.copy()
h,w = image.shape[:2]
for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2]
    if confidence > 0.5:
        # diagonal points 3: 7
        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        box = box.astype('int')
        pt1 = (box[0],box[1])
        pt2 = (box[2],box[3])
        # draw rectangle
        cv2.rectangle(image,pt1,pt2,(0,255,0),1)

        text = 'score : {:.0f} %'.format(confidence*100)
        cv2.putText(image,text,pt1,cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)

cv2.imshow('face detection',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Create Face Detection Function

In [16]:
def face_detection_dnn(img):
    # step-1: blob from image
    blob = cv2.dnn.blobFromImage(img,1,(300,300),(104,177,123),swapRB=True)
    # step-2: set blob as input
    face_detection_model.setInput(blob)
    # step-3: get the output
    detections = face_detection_model.forward()
    #  step-4: drawing boundng box on top of face detected
    image = img.copy()
    h,w = image.shape[:2]
    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2]
        if confidence > 0.5:
            # diagonal points 3: 7
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            box = box.astype('int')
            pt1 = (box[0],box[1])
            pt2 = (box[2],box[3])
            # draw rectangle
            cv2.rectangle(image,pt1,pt2,(0,255,0),1)

            text = 'score : {:.0f} %'.format(confidence*100)
            cv2.putText(image,text,pt1,cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)

    return image

In [17]:
img_detect = face_detection_dnn(img)

In [18]:
cv2.imshow('face detection',img_detect)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Real Time Face Detection with DNN

In [19]:
cap = cv2.VideoCapture(0)

face_detection_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt',
                                                './models/res10_300x300_ssd_iter_140000_fp16.caffemodel')

while True:
    ret, frame = cap.read()
    
    if ret == False:
        break
        
    img_detection = face_detection_dnn(frame)
    
    cv2.imshow('Real Time Face Detection with DNN',img_detection)
    if cv2.waitKey(1) == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()